## Imports

In [ ]:
# importing all relevant libraries, packages, and functionalities

import pandas as pd
import re
from textacy import preprocessing
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
# installing NLTK data for lemmatization

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
# importing the data set

df = pd.read_excel(r'Artificial Intelligence Companies.xlsx')

## Data Preparation

In [ ]:
# dropping all companies where Full Description is NaN and resetting index

df_clean = df[df['Full Description'].notna()].reset_index()

In [ ]:
# filtering the categories column of the data frame for specific, relevant categories

categories = ['Health', 'Care', 'Diagnostics', 'Biotechnology', 'Hospital', 'Medical', 'Pharmaceutical', 'Genetics']

df_clean['Category incl.'] = df_clean['Categories'].apply(lambda row: any(category in row for category in categories))
df_clean = df_clean.loc[df_clean['Category incl.'] == True].reset_index().drop(columns=['index', 'level_0'])

In [ ]:
# explicitly turning descriptions to type str

df_clean['Full Description'] = df_clean['Full Description'].apply(lambda row: str(row))

In [ ]:
# converting data frame column to a list called description_list

description_list = df_clean['Full Description'].tolist()

In [ ]:
# defining benchmark string

benchmark = 'Our AI-powered solutions address major challenges that are facing the healthcare field. Right now, the demand for diagnostic services is outpacing the supply of experts in the workforce. Developing solutions for managing this ever-increasing workload is a crucial task for the healthcare sector. Moreover, as the workload is growing, diagnostics and treatment are also becoming more complex. Diagnostic experts and physicians need a new set of tools that can handle large volumes of medical data quickly and accurately, allowing you to make more objective treatment decisions based on quantitative data and tailored to the needs of the individual patient. To provide this new toolset, we will need to draw on the power of artificial intelligence (AI).'

In [ ]:
# appending benchmark to description_list

description_list.append(str(benchmark))

In [ ]:
# defining a function called preprocessor, which takes a list of strings as input and returns a list of processed strings

def preprocessor(lst):
    processed_strings = [re.sub(r'\d+','', preprocessing.normalize_whitespace(preprocessing.remove_punctuation(lst[i].lower()))) for i in range(len(lst))]
    stopwords_removed = [remove_stopwords(processed_strings[i]) for i in range(len(processed_strings))]
    return stopwords_removed

In [ ]:
# processing strings to remove whitespace, punctuation, and numbers and converting case to lower

processed_strings_no_stopwords = preprocessor(description_list)

In [ ]:
# converting list of preprocessed strings to one-column data frame to prepare for lemmatization

processed_strings_df = pd.DataFrame(processed_strings_no_stopwords, columns=['strings'])

In [ ]:
# defining functions to perform lemmatization with appropriate POS tagging

lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()

# converts nltk tags to wordnet tags
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# takes a sentence string as input and returns the lemmatized sentence string
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [ ]:
# performing lemmatization and converting the result back into a list

processed_strings_df['lemmatized strings'] = processed_strings_df['strings'].apply(lambda x: lemmatize_sentence(x))
preprocessed_descriptions_final = processed_strings_df['lemmatized strings'].tolist()

## Jaccard Similarity Implementation

In [ ]:
# defining relevant functions for Jaccard Similarity (intersection and union)

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3

def union(lst1, lst2): 
    final_list = lst1 + lst2 
    return final_list 

def jaccard_similarity(lst1, lst2):
    s1 = lst1.split()
    s2 = lst2.split()
    return len(intersection(s1,s2)) / (len(union(s1,s2)) - len(intersection(s1,s2)))

In [ ]:
# calculate the similarity between Siemens and every other firm from the Crunchbase database 

body = processed_strings_no_stopwords[:]
similarity = [jaccard_similarity(benchmark, body[i]) for i in range(len(body))]

In [ ]:
# append similarity information to the firms' fundamental information

sim_df = pd.DataFrame(similarity, columns=['Jaccard Similarities'])
df_clean['Jaccard Similarities'] = sim_df['Jaccard Similarities']

In [ ]:
# rank the firms according to the similarity

df_clean = df_clean.sort_values(['Jaccard Similarities'], ascending=False)

In [ ]:
# exporting top 20 similar companies to Excel

df_clean.head(20).to_excel(r'Jaccard.xlsx')